## ETL API https://rickandmortyapi.com/api/ 

## Almacenamiento de datos AWS-S3

In [2]:
import boto3 as b3
import pandas as pd
import datetime as dt

from package import rickandmortyapi

### Variables del notebook.

In [3]:
aws_credentials = pd.read_csv("new_user_credentials.csv")
bucket_name     = "rickandmorty1046"
file            = 'data/db_character.csv'
currDate        = dt.datetime.now()

### Uso del paquete **rickandmorty** para extraer los datos.

In [4]:
db_characters = rickandmortyapi.extract_character(start_page = 1, end_pages = 40)
db_characters.to_csv(file)
db_characters.head()

,id,name,status,species,type,gender,origin,location,image,episode,url,created,page
0,1,Rick Sanchez,Alive,Human,,Male,"{'name': 'Earth (C-137)', 'url': 'https://rick...","{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,1
1,2,Morty Smith,Alive,Human,,Male,"{'name': 'unknown', 'url': ''}","{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z,1
2,3,Summer Smith,Alive,Human,,Female,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z,1
3,4,Beth Smith,Alive,Human,,Female,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/4,2017-11-04T19:22:43.665Z,1
4,5,Jerry Smith,Alive,Human,,Male,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/5,2017-11-04T19:26:56.301Z,1


### Conexión a **S3** y listado de buckets.

In [5]:
AWS_KEY_ID = aws_credentials['Access key ID'][0]
AWS_SECRET = aws_credentials['Secret access key'][0]

s3 = b3.client('s3', 
               aws_access_key_id = AWS_KEY_ID,
               aws_secret_access_key = AWS_SECRET)

buckets_resp = s3.list_buckets()

for bucket in buckets_resp["Buckets"]:
    print(bucket)

{'Name': 'elasticbeanstalk-us-east-2-003709122169', 'CreationDate': datetime.datetime(2022, 6, 6, 17, 19, 19, tzinfo=tzutc())}
{'Name': 'rickandmorty1046', 'CreationDate': datetime.datetime(2022, 6, 12, 2, 13, 8, tzinfo=tzutc())}


### Carga de datos a **S3**

In [6]:
s3.upload_file(Filename = file,
               Bucket   = bucket_name,
               Key      = 'db_character_{}.csv'.format(currDate))

In [7]:
response = s3.list_objects_v2(Bucket = bucket_name)

for obj in response["Contents"]:
    print(obj)

{'Key': 'db_character_2022-06-11 20:22:30.982890.csv', 'LastModified': datetime.datetime(2022, 6, 12, 2, 14, 4, tzinfo=tzutc()), 'ETag': '"efc66537c088efaa52a30de6b6d190e4"', 'Size': 317564, 'StorageClass': 'STANDARD'}
{'Key': 'db_character_2022-06-12 20:09:54.500863.csv', 'LastModified': datetime.datetime(2022, 6, 13, 1, 10, 39, tzinfo=tzutc()), 'ETag': '"efc66537c088efaa52a30de6b6d190e4"', 'Size': 317564, 'StorageClass': 'STANDARD'}
